In [ ]:
import boto3
import gzip
import requests
import shutil
import os
from zipfile import ZipFile

In [ ]:
# Test ZIP file download with unrestricted file
# This works
url = 'https://www.stats.govt.nz/assets/Uploads/Business-financial-data/Business-financial-data-December-2023-quarter/Download-data/business-financial-data-december-2023-quarter.zip'
r = requests.get(url)
with open('test.zip', 'wb') as f:
    f.write(r.content)

with ZipFile('test.zip', 'r') as f:
    f.extractall()

In [ ]:
# For now: Download restricted ZIP file manually
# TODO: Figure out how to login and download restricted files programmatically 
# Unzip 2023_CORE.zip programmatically
with ZipFile('./xml/2023_CORE.zip', 'r') as f:
    f.extractall('./xml/2023_CORE')

In [ ]:
# S3 setup
s3c = boto3.client('s3', region_name = 'us-east-1')
s3r = boto3.resource('s3', region_name = 'us-east-1')
bucket = s3c.create_bucket(Bucket = 'wos-bucket')

In [ ]:
# Test write-and-delete workflow
file = 'test.txt'
f = open(file, 'w')
f.write('Testing')
f = open(file, 'rb')
s3r.Bucket('wos-bucket').put_object(
          Key = 'test', # File name
          Body = f) # File
os.remove(file)

In [ ]:
# Write XML files to S3
# Ref: https://bito.ai/resources/unzip-gz-file-python-python-explained/
files = ['./xml/2023_CORE/WR_2023_20240112181857_CORE_0001.xml.gz',
         './xml/2023_CORE/WR_2023_20240112181857_CORE_0002.xml.gz', 
         './xml/2023_CORE/WR_2023_20240112181857_CORE_0003.xml.gz']

for file in files:
    xml = file[16:-3]
    with gzip.open(file, 'rb') as zipped:
       with open(xml, 'wb') as unzipped:
         shutil.copyfileobj(zipped, unzipped)
    body = open(xml, 'rb')
    s3r.Bucket('wos-bucket').put_object(
          Key = xml, # File name
          Body = body) # File
    # Delete files to make space
    os.remove(file)
    os.remove(xml)

In [ ]:
# Write schemas to S3
for _, _, files in os.walk('./Schemas'):
    for file in files:
        body = open('./Schemas/' + file, 'rb')
        s3r.Bucket('wos-bucket').put_object(
          Key = file,
          Body = body)


In [ ]:
# Write converter files to S3
# https://github.com/blackrock/xml_to_parquet
with open('./xml_to_parquet.py', 'rb') as f:
    s3r.Bucket('wos-bucket').put_object(
          Key = 'xml_to_parquet.py',
          Body = f)
    
with open('./convert_xml_to_parquet.py', 'rb') as f:
    s3r.Bucket('wos-bucket').put_object(
          Key = 'convert_xml_to_parquet.py',
          Body = f)
    
with open('./PurchaseOrder.xml', 'rb') as f:
    s3r.Bucket('wos-bucket').put_object(
          Key = 'PurchaseOrder.xml',
          Body = f)
    
with open('./PurchaseOrder.xsd', 'rb') as f:
    s3r.Bucket('wos-bucket').put_object(
          Key = 'PurchaseOrder.xsd',
          Body = f)

In [7]:
# Write nested demo data to S3
# https://jsoneditoronline.org/indepth/datasets/json-file-example/
with open('users_1k.json', 'rb') as f:
    s3r.Bucket('wos-bucket').put_object(
          Key = 'users_1k.json',
          Body = f)

ClientError: An error occurred (AccessDenied) when calling the PutObject operation: Access Denied